# Chapter 1 Results

**Objective:** Basic illustrative statistics to demonstrate the impact of model type, hyper-parameter optimisation, and evaluation metrics on document-model scoring

**Method**: We carry out the following experiments

 * For three datasets, for MoM/VB, MoM/Gibbs, LDA/VB, LDA/CVB, LDA/CVB0 for K in `{5, 10, 25, 50, 100, 200}` evaluate perplexity as document completion. Create multi-line trend plot. Persist
 * For three datasets, for HDP with three concentrations, evaluate perplexity as document completion. Create a second trend-plot, overlaid with LDA above
 * For three datasets, for LDA/VB for K in `{5, 10, 25, 50, 100, 200}` with hyperparam enabled, evaluate perplexity. Plot against LDA without hyper-param enabled.
 * For three datasets, for LDA/VB, for on K in `$BEST_K` with batch sizes of all, 1, 25, 100, ALL, plot held out perplexity every 10 iterations. 
 * For Reuters and 20News, for K in the usual range, generate scores according to the usual methods
 

**Outstanding Implementation Details:** Next steps

 * We need to be able to come up with a document-completion ScoringMethod.
     * This in turn requires us to throw an exception if `y` or `y_query_state` is supplied.
     * It also requires us to do the split in the `score()` method
 * If we assume the _training_ is the hardest part, then doing the querying for all the other cross-validation scores is easy
     * So change the workflow to use a custom split
     * and then write code to do the extra scores
     * and then write code to process them 
 * Is perplexity sensible. If it _sums_ then it's enormous. Probably best thing to do is to calculate the log-likelihood, and then have a separate function that for a given corpus sums the log-likelihood to get the overall corpus perplexity..
     * With the doc-completion split this will take some doing, do we just make an optimistic assumption that things cancel out?



# Prelude

## Imports

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as rd
import scipy as sp
import scipy.stats as stats
import scipy.sparse as ssp
import pathlib
import os
import sys
from IPython.display import display, Markdown
import importlib

import matplotlib.pyplot as plt

In [ ]:
import logging

_ = logging.getLogger()
logging.basicConfig(
    format='%(asctime)s %(levelname)-7s %(module)s::%(funcName)s() - %(message)s',
    level=logging.INFO
)

In [ ]:
%matplotlib inline

In [ ]:
sys.path.append(str(pathlib.Path.cwd().parent))

In [ ]:
from sidetopics.model.common import DataSet
import sidetopics.model.sklearn.lda_cvb as _lda_cvb
import sidetopics.model.sklearn as mytopics

importlib.reload(_lda_cvb)
importlib.reload(mytopics)

## Configuration

In [ ]:
RANDOM_SEED = 0xC0FFEE
rd.seed(RANDOM_SEED)

In [ ]:
ROOT_DIR = pathlib.Path(pathlib.Path.home().root)
WINDOWS_PARTITON = ROOT_DIR / 'media' / 'bfeeney' / 'Blade 15'
WINDOWS_HOME_DIR = WINDOWS_PARTITON / 'Users' / 'bryan'

GDRIVE_DIR = WINDOWS_HOME_DIR / 'Google Drive'
DATASET_DIR_ON_WIN_GDRIVE = GDRIVE_DIR / 'DatasetSSD'
DATASET_DIR_ON_MACOS_SSD = ROOT_DIR / 'Volumes' / 'DatasetSSD'

if DATASET_DIR_ON_MACOS_SSD.exists():
    DATASET_DIR = DATASET_DIR_ON_MACOS_SSD
else:
    DATASET_DIR = DATASET_DIR_ON_WIN_GDRIVE
assert DATASET_DIR.exists(), f"Cannot find dataset directory {DATASET_DIR}"
    
CLEAN_DATASET_DIR = DATASET_DIR / 'words-only'

T20_NEWS_DIR = CLEAN_DATASET_DIR / '20news4'
NIPS_DIR = CLEAN_DATASET_DIR / 'nips'
REUTERS_DIR = CLEAN_DATASET_DIR / 'reuters'

TRUMP_WEEKS_DIR = DATASET_DIR / 'TrumpDb'
NUS_WIDE_DIR = DATASET_DIR / 'NusWide'

CITHEP_DATASET_DIR = DATASET_DIR / 'Arxiv'
ACL_DATASET_DIR = DATASET_DIR / 'ACL' / 'ACL.100.clean'

In [ ]:
DTYPE = np.float32

In [ ]:
TOPIC_COUNTS = [5, 10, 25, 50, 100, 200]
BATCH_SIZES = [1, 10, 50, 100, 500, 100]

# DataSet Load

In [ ]:
t20_news = DataSet.from_files(words_file=T20_NEWS_DIR / 'words.pkl')
reuters = DataSet.from_files(words_file=REUTERS_DIR / 'words.pkl')
acl = DataSet.from_files(words_file=ACL_DATASET_DIR / 'words.pkl')
arxiv = DataSet.from_files(words_file=CITHEP_DATASET_DIR / 'words.pkl')
nips = DataSet.from_files(words_file=NIPS_DIR / 'words.pkl')

In [ ]:
t20_news.convert_to_dtype(DTYPE)
reuters.convert_to_dtype(DTYPE)
acl.convert_to_dtype(DTYPE)
arxiv.convert_to_dtype(DTYPE)
nips.convert_to_dtype(DTYPE)

# Maybe shuffle here in order to keep 

In [ ]:
def corpus_stats(dataset: DataSet, min_word_count: int = 25) -> str:
    quarts = np.percentile(a=dataset.words.sum(axis=1), q=[0, 25, 50, 75, 100]).astype(np.int32)
    quarts_str = ' | '.join(f'{q:,}' for q in quarts)
    
    doc_min_count = (dataset.doc_lens > min_word_count).sum()
    
    return f'{dataset.doc_count:,} | {doc_min_count:,} | {int(dataset.word_count):,} | {dataset.words.shape[1]} | {quarts_str}'


display(Markdown(f"""

| Dataset | Document Count | Document > 50 Count | Total Words | Vocabulary Size | DocLen (Min) | DocLen (25) | DocLen (50) | DocLen (75) | DocLen (Max) |
| ------- | -------------- | ------------------- | ----------- | --------------- | ------------ | ----------- | ----------- | ----------- | ------------ |
| Reuters-21578 | {corpus_stats(reuters)} |
| 20-News | {corpus_stats(t20_news)} |
| NIPS | {corpus_stats(nips)} |
| ACL | {corpus_stats(acl)} |
| Arxiv | {corpus_stats(arxiv)} |

"""))

The thing to emphasise here is that we're deliberately chose a mix of datasets with small document lengths and large document lengths to look into overparameterisation.

# Plot 1: Perplexity Across K and Datasets

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
X = reuters.words

In [ ]:
gmodel = GridSearchCV(
    estimator=mytopics.TopicModel(
        kind='LDA_VB',
        n_components=10,
        seed=RANDOM_SEED,
        default_scoring_method=mytopics.ScoreMethod.DocCompletionPerplexityPoint
    ),
    cv=KFold(n_splits=5, shuffle=True, random_state=rd.RandomState(seed=0xC0FFEE)),
    n_jobs=14,
    param_grid={
        'kind': ['MOM_VB', 'MOM_GIBBS', 'LDA_GIBBS', 'LDA_CVB', 'LDA_CVB0']
    }
)

gmodel.fit(X)

In [ ]:
import pickle as pkl

model_state_dir = pathlib.Path.cwd() / 'saved_models'
if not model_state_dir.exists():
    model_state_dir.mkdir()

In [ ]:
with open(model_state_dir / 'topic-model-general.pkl', 'wb') as f:
    pkl.dump(gmodel, f)

In [ ]:
with open(model_state_dir / 'topic-model-general.pkl', 'rb') as f:
    gmodel_restored = pkl.load(f)

In [ ]:
gmodel.cv_results_

In [ ]:
gmodel_restored.cv_results_

In [ ]:
gmodel = GridSearchCV(
    estimator=mytopics.WrappedSckitLda(
        n_components=10,
        seed=RANDOM_SEED,
        default_scoring_method=mytopics.ScoreMethod.DocCompletionPerplexityPoint
    ),
    cv=5,
    n_jobs=14,
    param_grid={
        'kind': ['MOM_VB'],
        'n_components': TOPIC_COUNTS
    }
)

gmodel.fit(X)
plt.plot(TOPIC_COUNTS, gmodel.cv_results_['mean_test_score'], 's-')

In [ ]:
for dataset in [t20_news, reuters, acl, arxiv, nips]:
    X = dataset.words

# Appendix

## Issue 1: MoM vs LDA

In [ ]:
train, test = reuters.cross_valid_split(test_fold_id=0, num_folds=5)

In [ ]:
test, valid = test.doc_completion_split()

In [ ]:
from gensim.sklearn_api import HdpTransformer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
LatentDirichletAllocation.score?

## How are we evaluating this?

First off, the `LatentDirichletAllocation` class in `sklearn` will use the variational bound as an approximation of the log likeihood with a give set of doc-to-topic distributions. There's no doc-completion thing here. 

This is the basis of `score()`, which internally calculates the "unnormalized" topic distribution of the documents, then uses the variational bound to approximate the log likelihood; this in turn is the basis of `perplexity()`.

What did Hannah Wallach say?

 * Well she's thinking of T topics, and I guess ? words so here component distribution is  $\Phi \in \mathbb{R}^{T \times ?}$ with prior $\text{Dir}(\phi_t; \beta \boldsymbol{n})$
 * For each of the $D$ documents there's a topic distribution $\theta_d$ with prior $\text{Dir}(\theta_d; \alpha \boldsymbol{m})$

Finally, she notes the Polya identity, allowing the marginalisation of most parameters.

She then moves out into how to evaluate the probability of some held out documents $W$ given some training documents $W'$ which is

$$
p(W | W') = \int d\Phi d\alpha d\boldsymbol{m}
             \text{ } p(W | \Phi, \alpha, \boldsymbol{m}) \text{ } p(\Phi, \alpha, \boldsymbol{m}|W')
$$

The thing to note here is she has already margnalised out $\Theta$ for the new documents. She assumes you learn the "global" parameters -- priors and component distribution -- and then fix these and use them to evaluate the new documents

> So we have to think about what we're doing here. A mixture model is a good case. You can just directly evaluate the log likelihood $p(w|\alpha, \Phi) = \sum_k p(w | \phi_k)p(z=k|\alpha)$. Or you can determine the posterior over clusters and use that to evaluate... except that it doesn't decompose $p(w|\alpha, \Phi) = \sum_k p(w, z=k|\alpha, \Phi) = p(z=k|w, \alpha, \Phi)p(w|\ldots)$. But it seems obvious to see how well you can "explain" documents: this is what doc-completion does. Hence it should be introduced in the clustering section. It's also a good metric to use if you want to consider the predictive ability to, e.g. predict hashtags.

Now either way, you have to make a choice about your parameters. Are you using the _distribution_ over the parameters, or are you just taking a point estimate?

1. Drawing samples from the parameter posterior and taking an average to evaluate the integral, i.e.  $\mathbb{E}_{p(\Phi, \alpha, \boldsymbol{m}|W')}\left[ p(W | \Phi, \alpha, \boldsymbol{m}) \right]$. 
    * Stick a log in that expectation and you can start thinking about a variational approximation.
2. Taking a point estimate of -- I guess $\Phi, \alpha, \boldsymbol{m}$ -- and then use that to approximate

The paper is concerned with point estimates. So where's the uncertainty.... Apparently its in $p(\boldsymbol{w}_d | \alpha \boldsymbol{m}, \Phi)$

The next thing is that we've marginalised out $\theta$ for each of the inference documents. We need this too. If you hold $\Phi$ fixed (and so let it be found by any inference method), you can use Gibbs sampling to quickly get a distribution over $z$ and thereby, $\theta$.

 * This is used by many methods she describes, being: FIXME
 * There are other methods that do not require this, being: FIXME
 


### Estimating $p(w|\Phi, a \boldsymbol{m})$



#### Using Importance Sampling


Hence there are two options:

Directly sample $\theta \sim Dir(\alpha \boldsymbol{m})$ and average over all settings. But importance sampling doesn't work well in high-dimensions: it has high-variance, indeed, infinite variance with real-values high-dim values.

The other is to choose a proposal distribution and weight such samples in the usual importance-sampling way. The proposal distribution is in fact a method for evaluating the posterior $p(z|w, \alpha \boldsymbol{m}, \Phi)$

$$
\theta^0 \propto \left(\alpha \boldsymbol{m}\right) \text{.* } \Phi_{\cdot, w_{n}} 
$$

Which is just the prior over topics and the probability of words under each topic, i.e. $p(z = k| w, \Phi, \alpha \boldsymbol{m}) \propto p(w|Phi, z=k)p(z=k| \alpha \boldsymbol{m})$

To draw samples, simply iterate
$$
\begin{align*}
\text{for }& s = 0 \ldots S \\
 & z_n^{(s)} \sim \text{Mul}(\theta^{(s)}, 1) \\
 & \theta^{(s+1)} \propto \left(\alpha \boldsymbol{m} + \sum_{n' \neq n} \theta^{(s)} \text{.* } \boldsymbol{\bar{z}}_{n'}\right) \Phi_{\cdot, w_{n}}
\end{align*}
$$

(Recall that in more normal notation $\alpha \boldsymbol{m} = \boldsymbol{\alpha}$ and parameterises the prior. Also $z_n$ is the scalar and $\bar{\boldsymbol{z}}_n$ is the indicator vector.

#### Use the Harmonic Mean

Use Gibbs sampling to get a _posterior_ distribution over $z_n^s$.

Then instead of using that to materlise an estimate of $\theta$ (WHY), use it directly to figure out $p(w | \alpha \boldsymbol{m}, \Phi)$